<a href="https://colab.research.google.com/github/kannavdhawan/fake_news_challenge/blob/master/evaluations_lstm_bi_lstm_all_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

### Importing predicted test data csv's

In [4]:
cnn_50=pd.read_csv('/content/drive/My Drive/fnc-1/cnn_50.csv')
cnn_100=pd.read_csv('/content/drive/My Drive/fnc-1/cnn_100.csv')
cnn_150=pd.read_csv('/content/drive/My Drive/fnc-1/cnn_150.csv')

lstm_50=pd.read_csv('/content/drive/My Drive/fnc-1/lstm_50.csv')
lstm_100=pd.read_csv('/content/drive/My Drive/fnc-1/lstm_100.csv')
lstm_150=pd.read_csv('/content/drive/My Drive/fnc-1/lstm_150.csv')

bi_lstm_50=pd.read_csv('/content/drive/My Drive/fnc-1/bi_lstm_50.csv')
bi_lstm_100=pd.read_csv('/content/drive/My Drive/fnc-1/bi_lstm_100.csv')
bi_lstm_150=pd.read_csv('/content/drive/My Drive/fnc-1/bi_lstm_150.csv')

# w2v embeddings

w2v_cnn_50=pd.read_csv('/content/drive/My Drive/fnc-1/w2v_cnn_50.csv')
w2v_lstm_50=pd.read_csv('/content/drive/My Drive/fnc-1/w2v_lstm_50.csv')
w2v_bi_lstm_50=pd.read_csv('/content/drive/My Drive/fnc-1/w2v_bi_lstm_50.csv')

### Importing Real labels

In [7]:
real_bodies=pd.read_csv('/content/drive/My Drive/fnc-1/competition_test_bodies.csv')
real_stances=pd.read_csv('/content/drive/My Drive/fnc-1/competition_test_stances.csv')

In [11]:
real_stances.head(3)

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated


In [12]:
cnn_50.head(3)

,Headline,Body ID,Stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated


In [14]:
#Adapted from https://github.com/FakeNewsChallenge/fnc-1/blob/master/scorer.py
#Original credit - @bgalbraith

LABELS = ['agree', 'disagree', 'discuss', 'unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm


def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


def report_score(actual,predicted):
    score,cm = score_submission(actual,predicted)
    best_score, _ = score_submission(actual,actual)

    print_confusion_matrix(cm)
    print("Score: " +str(score) + " out of " + str(best_score) + "\t("+str(score*100/best_score) + "%)")
    return score*100/best_score


In [33]:
from sklearn.metrics import f1_score

def calculate_f1_scores(y_true, y_predicted):
    
    f1_macro = f1_score(y_true, y_predicted, average='macro')
    f1_classwise = f1_score(y_true, y_predicted, average=None, labels=["agree", "disagree", "discuss", "unrelated"])

    resultstring = "F1 macro: {:.3f}".format(f1_macro * 100) + "% \n"
    resultstring += "F1 agree: {:.3f}".format(f1_classwise[0] * 100) + "% \n"
    resultstring += "F1 disagree: {:.3f}".format(f1_classwise[1] * 100) + "% \n"
    resultstring += "F1 discuss: {:.3f}".format(f1_classwise[2] * 100) + "% \n"
    resultstring += "F1 unrelated: {:.3f}".format(f1_classwise[3] * 100) + "% \n"
    return resultstring

In [34]:
from sklearn.metrics import classification_report
def class_repo(labels_test,preds_test):
    eval_report = classification_report(labels_test, preds_test)
    print('Test report', eval_report)

## Evaluating CNN 50

In [16]:
report_score(real_stances['Stance'],cnn_50['Stance'])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    65     |    449    |    397    |    992    |
-------------------------------------------------------------
| disagree  |    30     |    132    |    101    |    434    |
-------------------------------------------------------------
|  discuss  |    153    |    551    |   1505    |   2255    |
-------------------------------------------------------------
| unrelated |    310    |   1323    |   1751    |   14965   |
-------------------------------------------------------------
Score: 5863.5 out of 11651.25	(50.32507241712263%)


50.32507241712263

In [36]:
print(calculate_f1_scores(real_stances['Stance'],cnn_50['Stance']))

F1 macro: 32.797% 
F1 agree: 5.282% 
F1 disagree: 8.376% 
F1 discuss: 36.627% 
F1 unrelated: 80.903% 



In [48]:
print(class_repo(real_stances['Stance'],cnn_50['Stance']))

Test report               precision    recall  f1-score   support

       agree       0.12      0.03      0.05      1903
    disagree       0.05      0.19      0.08       697
     discuss       0.40      0.34      0.37      4464
   unrelated       0.80      0.82      0.81     18349

    accuracy                           0.66     25413
   macro avg       0.34      0.34      0.33     25413
weighted avg       0.66      0.66      0.65     25413

None


## Evaluating CNN 100

In [17]:
report_score(real_stances['Stance'],cnn_100['Stance'])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    21     |    330    |    352    |   1200    |
-------------------------------------------------------------
| disagree  |    11     |    78     |    108    |    500    |
-------------------------------------------------------------
|  discuss  |    36     |    402    |   1089    |   2937    |
-------------------------------------------------------------
| unrelated |    117    |    882    |   1333    |   16017   |
-------------------------------------------------------------
Score: 5502.0 out of 11651.25	(47.22240102993241%)


47.22240102993241

In [37]:
print(calculate_f1_scores(real_stances['Stance'],cnn_100['Stance']))

F1 macro: 30.081% 
F1 agree: 2.011% 
F1 disagree: 6.530% 
F1 discuss: 29.649% 
F1 unrelated: 82.132% 



In [50]:
print(class_repo(real_stances['Stance'],cnn_100['Stance']))

Test report               precision    recall  f1-score   support

       agree       0.11      0.01      0.02      1903
    disagree       0.05      0.11      0.07       697
     discuss       0.38      0.24      0.30      4464
   unrelated       0.78      0.87      0.82     18349

    accuracy                           0.68     25413
   macro avg       0.33      0.31      0.30     25413
weighted avg       0.64      0.68      0.65     25413

None


## Evaluating CNN 150

In [18]:
report_score(real_stances['Stance'],cnn_150['Stance'])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    49     |    259    |    396    |   1199    |
-------------------------------------------------------------
| disagree  |    21     |    99     |    106    |    471    |
-------------------------------------------------------------
|  discuss  |    90     |    288    |   1424    |   2662    |
-------------------------------------------------------------
| unrelated |    250    |    984    |   2386    |   14729   |
-------------------------------------------------------------
Score: 5544.25 out of 11651.25	(47.585023066194616%)


47.585023066194616

In [38]:
print(calculate_f1_scores(real_stances['Stance'],cnn_150['Stance']))

F1 macro: 30.985% 
F1 agree: 4.237% 
F1 disagree: 8.509% 
F1 discuss: 32.452% 
F1 unrelated: 78.744% 



In [51]:
print(class_repo(real_stances['Stance'],cnn_150['Stance']))

Test report               precision    recall  f1-score   support

       agree       0.12      0.03      0.04      1903
    disagree       0.06      0.14      0.09       697
     discuss       0.33      0.32      0.32      4464
   unrelated       0.77      0.80      0.79     18349

    accuracy                           0.64     25413
   macro avg       0.32      0.32      0.31     25413
weighted avg       0.63      0.64      0.63     25413

None


## Evaluating LSTM 50 

In [19]:
report_score(real_stances['Stance'],lstm_50['Stance'])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    33     |    336    |    267    |   1267    |
-------------------------------------------------------------
| disagree  |    12     |    119    |    82     |    484    |
-------------------------------------------------------------
|  discuss  |    18     |    337    |   1545    |   2564    |
-------------------------------------------------------------
| unrelated |    82     |    643    |   1385    |   16239   |
-------------------------------------------------------------
Score: 6019.75 out of 11651.25	(51.6661302435361%)


51.6661302435361

In [39]:
print(calculate_f1_scores(real_stances['Stance'],lstm_50['Stance']))

F1 macro: 34.444% 
F1 agree: 3.223% 
F1 disagree: 11.163% 
F1 discuss: 39.907% 
F1 unrelated: 83.485% 



In [52]:
print(class_repo(real_stances['Stance'],lstm_50['Stance']))

Test report               precision    recall  f1-score   support

       agree       0.23      0.02      0.03      1903
    disagree       0.08      0.17      0.11       697
     discuss       0.47      0.35      0.40      4464
   unrelated       0.79      0.89      0.83     18349

    accuracy                           0.71     25413
   macro avg       0.39      0.35      0.34     25413
weighted avg       0.67      0.71      0.68     25413

None


## Evaluating LSTM 100 

In [20]:
report_score(real_stances['Stance'],lstm_100['Stance'])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     0     |    281    |    348    |   1274    |
-------------------------------------------------------------
| disagree  |     0     |    90     |    94     |    513    |
-------------------------------------------------------------
|  discuss  |     0     |    446    |   1375    |   2643    |
-------------------------------------------------------------
| unrelated |     0     |   1129    |   1665    |   15555   |
-------------------------------------------------------------
Score: 5646.0 out of 11651.25	(48.45831992275507%)


48.45831992275507

In [40]:
print(calculate_f1_scores(real_stances['Stance'],lstm_100['Stance']))

F1 macro: 30.644% 
F1 agree: 0.000% 
F1 disagree: 6.810% 
F1 discuss: 34.609% 
F1 unrelated: 81.155% 



In [53]:
print(class_repo(real_stances['Stance'],lstm_100['Stance']))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test report               precision    recall  f1-score   support

       agree       0.00      0.00      0.00      1903
    disagree       0.05      0.13      0.07       697
     discuss       0.39      0.31      0.35      4464
   unrelated       0.78      0.85      0.81     18349

    accuracy                           0.67     25413
   macro avg       0.30      0.32      0.31     25413
weighted avg       0.63      0.67      0.65     25413

None


## Evaluating LSTM 150

In [21]:
report_score(real_stances['Stance'],lstm_150['Stance'])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     1     |    455    |    432    |   1015    |
-------------------------------------------------------------
| disagree  |     3     |    174    |    158    |    362    |
-------------------------------------------------------------
|  discuss  |     4     |    486    |   1538    |   2436    |
-------------------------------------------------------------
| unrelated |    17     |   1379    |   1795    |   15158   |
-------------------------------------------------------------
Score: 5887.0 out of 11651.25	(50.526767514215216%)


50.526767514215216

In [41]:
print(calculate_f1_scores(real_stances['Stance'],lstm_150['Stance']))

F1 macro: 32.229% 
F1 agree: 0.104% 
F1 disagree: 10.906% 
F1 discuss: 36.676% 
F1 unrelated: 81.233% 



In [54]:
print(class_repo(real_stances['Stance'],lstm_150['Stance']))

Test report               precision    recall  f1-score   support

       agree       0.04      0.00      0.00      1903
    disagree       0.07      0.25      0.11       697
     discuss       0.39      0.34      0.37      4464
   unrelated       0.80      0.83      0.81     18349

    accuracy                           0.66     25413
   macro avg       0.33      0.36      0.32     25413
weighted avg       0.65      0.66      0.65     25413

None


## Evaluating Bi-LSTM 50

In [22]:
report_score(real_stances['Stance'],bi_lstm_50['Stance'])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    39     |    318    |    309    |   1237    |
-------------------------------------------------------------
| disagree  |    14     |    118    |    86     |    479    |
-------------------------------------------------------------
|  discuss  |    31     |    341    |   1701    |   2391    |
-------------------------------------------------------------
| unrelated |    43     |    639    |   1327    |   16340   |
-------------------------------------------------------------
Score: 6217.75 out of 11651.25	(53.365518721167255%)


53.365518721167255

In [42]:
print(calculate_f1_scores(real_stances['Stance'],bi_lstm_50['Stance']))

F1 macro: 35.595% 
F1 agree: 3.842% 
F1 disagree: 11.169% 
F1 discuss: 43.134% 
F1 unrelated: 84.235% 



In [55]:
print(class_repo(real_stances['Stance'],bi_lstm_50['Stance']))

Test report               precision    recall  f1-score   support

       agree       0.31      0.02      0.04      1903
    disagree       0.08      0.17      0.11       697
     discuss       0.50      0.38      0.43      4464
   unrelated       0.80      0.89      0.84     18349

    accuracy                           0.72     25413
   macro avg       0.42      0.37      0.36     25413
weighted avg       0.69      0.72      0.69     25413

None


## Evaluating Bi-LSTM 100

In [23]:
report_score(real_stances['Stance'],bi_lstm_100['Stance'])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    41     |    333    |    450    |   1079    |
-------------------------------------------------------------
| disagree  |    10     |    85     |    99     |    503    |
-------------------------------------------------------------
|  discuss  |    37     |    362    |   2043    |   2022    |
-------------------------------------------------------------
| unrelated |    82     |    599    |   1278    |   16390   |
-------------------------------------------------------------
Score: 6589.25 out of 11651.25	(56.55401780924794%)


56.55401780924794

In [44]:
print(calculate_f1_scores(real_stances['Stance'],bi_lstm_100['Stance']))

F1 macro: 36.666% 
F1 agree: 3.956% 
F1 disagree: 8.189% 
F1 discuss: 49.028% 
F1 unrelated: 85.491% 



In [56]:
print(class_repo(real_stances['Stance'],bi_lstm_100['Stance']))

Test report               precision    recall  f1-score   support

       agree       0.24      0.02      0.04      1903
    disagree       0.06      0.12      0.08       697
     discuss       0.53      0.46      0.49      4464
   unrelated       0.82      0.89      0.85     18349

    accuracy                           0.73     25413
   macro avg       0.41      0.37      0.37     25413
weighted avg       0.70      0.73      0.71     25413

None


## Evaluating Bi-LSTM 150

In [24]:
report_score(real_stances['Stance'],bi_lstm_150['Stance'])

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    34     |    207    |    305    |   1357    |
-------------------------------------------------------------
| disagree  |    24     |    54     |    103    |    516    |
-------------------------------------------------------------
|  discuss  |    51     |    209    |   1614    |   2590    |
-------------------------------------------------------------
| unrelated |    75     |    387    |   1136    |   16751   |
-------------------------------------------------------------
Score: 6114.5 out of 11651.25	(52.47934770947323%)


52.47934770947323

In [45]:
print(calculate_f1_scores(real_stances['Stance'],bi_lstm_150['Stance']))

F1 macro: 34.310% 
F1 agree: 3.258% 
F1 disagree: 6.950% 
F1 discuss: 42.351% 
F1 unrelated: 84.680% 



In [57]:
print(class_repo(real_stances['Stance'],bi_lstm_150['Stance']))

Test report               precision    recall  f1-score   support

       agree       0.18      0.02      0.03      1903
    disagree       0.06      0.08      0.07       697
     discuss       0.51      0.36      0.42      4464
   unrelated       0.79      0.91      0.85     18349

    accuracy                           0.73     25413
   macro avg       0.39      0.34      0.34     25413
weighted avg       0.68      0.73      0.69     25413

None


In [ ]:
# # Replacing the numeric stances with the original stances
# pred=[]
# for i in preds_test:
#   if i==0:
#     pred.append('agree')
#   elif i==1:
#     pred.append('disagree')
#   elif i==2:
#     pred.append('discuss')
#   elif i==3:
#     pred.append('unrelated')